# Building your own menus for PyGêBR

PyGêBR comes with many _menus_ for the common seismic-processing packages Madagascar and Seismic Un*x, as well as for some stand-alone tools. But it is not limited to those. You can integrate new programs to PyGêBR, even your own programs, by building _menus_ to them.

## Introduction

**PyGêBR** represents command-line programs and their chaining to compose processing flows. For example, consider the command line below, in which the program `sfscale` is run.

`sfscale <in.rsf >out.rsf axis=0 pclip=100. dscale=1.5`

In this line, `sfscale` is the _executable_.


`<in.rsf` informs that the file `in.rsf` is being feed in through the standart input, from which the program reads data. `>out.rsf` informs that data written by the program in the standard output will be saved in file `out.rsf`.

The rest of the elements in the command linea are _parameters_ which tune the program's behavior.

`axis` is a parameter that asks for an integer value (`0` in the line above). The parameters `pclip` and `dscale` both ask for real values. The text used to define these parameters (`axis=`, `pclip=` e `dscale=`) are their _keywords_. To PyGêBR, a keyword has all characters required for parameter definition in the command line. 

## Parameters

Beyond numerical valued parameters (real or integers), PyGêBR supports other types of parameters. Below you see the full list of supported parameters types.

 - **integer**: A parameter that expects an integer value as argument.  
   Ex.: `ns=10`
 - **range**: A parameter that expects a numerical value as argument within a finite interval (with minimum and maximum).  
   Ex.: `month=10`
 - **integers**: A parameter that expects a list of integer values as argument.  
   Ex.: `np=5,4,8,10`
 - **float**: A parameter that expects a real value as argument.  
   Ex.: `dx=2.5`
 - **floats**: A parameter that expects a list of real values as argument.  
   Ex.: `vel=1.5,2.2,4.5` 
 - **string**: A parameter that expects a text as argument.  
   Ex.: `label="Distância (km)"`
 - **strings**: A parameter that expects a list of texts as argument.  
   Ex.: `prefixes="Anti,Pos"`
 - **flag**: A boolean parameter that may or may not be in the command-line.  
   Ex.: `--verbose`
 - **file**: A parameter that expects a file name as argument.  
   Ex.: `--export=image.png`
 - **path**: A parameter that expects a path in the filesystem as argument.  
   Ex.: `--tmpdir=/tmp`
 - **enum**: A parameter that expect as argument a value from within a list of predefined possible values.  
   Ex.: `interpolation=linear` (where the options would be `linear`, `quadratic` or `cubic`)
 - **section**: Not really a program parameter, but rather a _meta parameter_ used to better organize parameters in the interface. After adding a section parameter, all subsequent parameters will be grouped together within the section.
  

## Program

PyGêBR defines a class to represent programs. To specify a program, some information must be provided. They are:

 - **executable**: the name of the executable command for the program (`sfscale`, in the example above).
 - **title**: short title used to present the program to users.
 - **description**: one-line sentence to state programs purpose.
 - **url**: a URL where more information about the program can be found.
 - **authors**: list of program's authors.
 - **stdin**: flag indicating whether the program reads data from standard in.
 - **stdout**: flag indicating whether the program writes data to standard output.
 - **stderr**: flag indicating whether the program writes data to standard error.

With the program created, it remains to add parameters to it.

## Menu

_Menus_ are used to gather programs and make them available to build processing flows. Most menus contain only one program, but sometimes it is useful to build a menu with two or more programs, which are usually employed in that particular way. To create a menu, you have to inform:

 - **title**: Short text to refer the menu.
 - **description**: one-line sentence describing the menu's purpose.
 - **authors**: list of the menu's authors.
 - **tags**: list of keywords used to classify the menu, to allow searching for the menu.
 
With the menu created, it remains to add programs to it.

# Building a menu step by step

In [ ]:
# Import all modules from PyGêBR

from pygebr import *

## Credits

We always try to make clear who are the authors of each program, menu and flow. You, as the menus creator, should be granted for that.

In [ ]:
# Create "you" to be used as the menu's author

you = Person(name="John Doe",
             email="john@email.com",
             institution="University Somewhere",
             homepage="https://www.your.page.com/")

# The credit holder for Madagascar program

rsf = Person(name="Madagascar",
             institution="University of Texas at Austin",
             homepage="https://www.reproducibility.org/")

 ## The program

In this example, we will construct a menu to hold only the `sfscale` program.

In [ ]:
# Create a representation for program sfscale

# To be able to insert this program into processing flows
# it is necessary to inform how the program deals with
# the system standard input and output. In this particular
# case, sfscale reads from standard input and writes to
# standard output.

title = "SF Scale"
desc = "Scale RSF data"

prog = Prog(title=title,           # title for the program
            description=desc,      # short description
            executable="sfscale",  # executable
            url="https://reproducibility.org/wiki/Guide_to_madagascar_programs#sfscale",
            authors=rsf,           # program's author
            # states that the program reads from the standar input
            stdin=True,    # that's the default and may be omitted
            # stats that the program writes to the standard output
            stdout=True,   # that's the default and may be omitted
            # stats that the program writes to the standard error
            stderr=True    # that's the default and may be omitted
           )

Let us now define each of sfscale's parameters.

In [ ]:
# Define the integer parameter axis, which is
# passed as axis=2 in the command line, for example.

par = Param(ptype="integer",   # parameter type
            # keyword for the parameter
            keyword="axis=",        
            # One-line description used to display this parameter in the UI
            description="Scale by maximum in the dimensions up to this axis", 
            # Informs that this parameter may be omited
            required=False,
            # Informs that this parameter may be defined only once
            multiple=False,
            # Defines a default value to be used in UI
            default=0
           )

# Add the parameter to the program object.

prog.parameter_add(par)

In [ ]:
# Define a new parameter of float type and add it
# directly this time.

prog.parameter_add(Param(ptype="float",
                         keyword="dscale=",
                         description="Scale factor",
                         required=False,
                         multiple=False,
                         default=1.0)
                  )

In [ ]:
# Define a parameter of range type. In this case, it isn't
# possible to add it directly, since some parameter's
# attributes can't be informed at the moment the parameter
# is declared.

par = Param(ptype="range",
            keyword="pclip=",
            description="Data clip percitile",
            required=False,
            multiple=False,
            default=100)

# Define the interval where for acceptable values for
# this parameter. Also informs the step used to increment
# or decrement the parameter's value in the UI, as well as
# how many decimal digits should be presented.

par.range([0,100], vinc=0.5, vdigits=2)

# Add the parameter to the program object.

prog.parameter_add(par)

All parameters that control `sfscale` behavior have be declared and added to program object, which is now done.

## The menu, at last

In [ ]:
# As this menu will have only one program, it is usual employ the same
# title and description used for the program. The author however isn't
# the same. You can inform a list of tags used to classify the menu,
# for indexing purposes.

menu = Flow(title=title,
            description=desc,
            authors=you,
            tags=['rsf','madagascar'])

# Add the program to the menu object.

menu.program_add(prog)

In [ ]:
# Show a report describing the menu

menu.dump(verbose=True,setonly=False)

In [ ]:
# Finally, save the menu

menu.save("/tmp/sfscale.json")

Now the file `sfscale.json` is a menu for program  `sfscale` and could be used by `LoadFlow` command to build processing flows. Note that `LoadFlow` looks for menus in some directories defined in the PyGêBR setup. Therefore, to be able to find your menu, save it into one of those directories.

In [ ]:
# To discover where `LoadFlow` looks for menus, run

setup = Setup()
print(setup)